# Python для анализа данных

## API


## API YouTube

+ [Инструкция](https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb) по получению ключа API YouTube
+ Официальная [документация](https://developers.google.com/youtube/v3/docs) к API YouTube
+ Статья на Хабр. [Что такое API?](https://habr.com/ru/post/464261/)
+ Статья на Хабр. [Что такое JSON](https://habr.com/ru/post/554274/)

API (Application programming interface) — это контракт, который предоставляет программа. Иными словами, это функции от разработчика, которые предоставлены в наше использование и распоряжение.

Сегодня попробуем потыкаться в API YouTube и вытащить какую-нибудь информацию. Но для этого нам нужен собственно доступ к API (ключ). Как его получить, можно узнать в [инструкции](https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb).

In [25]:
# !pip install google-api-python-client
import googleapiclient.discovery as api

In [26]:
# Поскольку API-ключи привязаны к личным аккаунтам и хранят эту связь, не стоит выкладывать в открытый доступ ваш ключ.
# Я создал отдельный файл youtube_key.py, где задал константу YT_API_KEY и положил свой токен в виде строки.
# Получить свой токен: https://sphenoid-aluminum-1ba.notion.site/API-YouTube-a1e2552472c246d295b6f1d1dbce5dcb

from youtube_key import *
key = YT_API_KEY
#YT_API_KEY = 'mykey' -- строка в файле

In [27]:
# key = 'ВАШ КЛЮЧ'
youtube = api.build('youtube', 'v3', developerKey = key)

Далее используем запрос, который предлагается нам в самой документации. Этот тот момент, где вам не нужно задумываться о том, почему метод называется так и почему он хочет определенные параметры — просто так придумал разработчик.

### Видео по опеределенному запросу

Например, в самой [документации](https://developers.google.com/youtube/v3/docs/search/list?hl=en) про параметр `part` сказано Set the parameter value to snippet.

`maxResults` — сколько видео (максимально можно 50, но, спойлер, это ограничение можно обойти) \
`q` — запрос, который мы ищем \
`type` — пока мы ищем видео, но можно и каналы \
`order` — по релевантности (его можно убрать) \
Это все наш запрос с помощью метода .`search()`. Выгружаем его командой `.execute()`.

In [ ]:
dir(youtube)

In [29]:
type(youtube)

googleapiclient.discovery.Resource

In [30]:
request_search = youtube.search().list(
    part = "snippet",
    maxResults = 10,
    q = "API with Python",
    type = 'video',
    order = 'relevance')

print(request_search)

response_search = request_search.execute()
response_search

{'kind': 'youtube#searchListResponse',
 'etag': 'x9l4KA6lBznAa4N-4E_g133XQ-8',
 'nextPageToken': 'CAoQAA',
 'regionCode': 'TW',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'gYeE34BNFK5iLDPvOTz1kKZcaug',
   'id': {'kind': 'youtube#video', 'videoId': 'zsYIw6RXjfM'},
   'snippet': {'publishedAt': '2023-05-23T13:00:53Z',
    'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
    'title': 'Create A Python API in 12 Minutes',
    'description': 'Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7 Are you learning programming and ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/zsYIw6RXjfM/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/zsYIw6RXjfM/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/zsYIw6RXjfM/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'cha

Это просто словарь! А с ними вы работать уже умеете.

К слову, использование этого API **бесплатно**, но не бесконечно. Есть определенный лимит для одного ключа —10.000 у.е в день (например, запрос видео выше стоил нам 100 у.е). Подробнее об этом можно почитать в самой документации. Такое придумано в API YouTube (для других API может быть что-то свое или вообще не быть лимита).

Посмотрим, какие тут есть ключи:

In [31]:
response_search.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'regionCode', 'pageInfo', 'items'])

Из них нас интересуют только `nextPageToken`, `pageInfo`, `items`.

`nextPageToken` — адрес следующей страницы нашего поиска; до тех пор пока он есть в выдаче, мы можем собирать данные. \
`pageInfo` — сколько всего результатов и сколько всего результатов по нашему запросу. Если всего результатов очень много, то все вы не получите (так уж устроен API YouTube); можно исхитриться и в цикле делать запрос от и до определенной даты. \
`items` — все видео, которые мы нашли (у нас их десять).

In [32]:
print(type(response_search['items'])) # это список!
print(len(response_search['items']))  # наши десять видео

<class 'list'>
10


Вытащим самое первое видео и посмотрим, что там.

In [33]:
response_search['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'gYeE34BNFK5iLDPvOTz1kKZcaug',
 'id': {'kind': 'youtube#video', 'videoId': 'zsYIw6RXjfM'},
 'snippet': {'publishedAt': '2023-05-23T13:00:53Z',
  'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
  'title': 'Create A Python API in 12 Minutes',
  'description': 'Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7 Are you learning programming and ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/zsYIw6RXjfM/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/zsYIw6RXjfM/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/zsYIw6RXjfM/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Tech With Tim',
  'liveBroadcastContent': 'none',
  'publishTime': '2023-05-23T13:00:53Z'}}

In [34]:
response_search['items'][0].keys()

dict_keys(['kind', 'etag', 'id', 'snippet'])

Тут нас интересуют ключи `id` и `snippet`.

* в `id` записано `'videoId'` (оно нам нужно, чтобы вытащить информацию о конкретном видео, например, статистику по лайкам или комментарии)
* `snippet` содержит основную информацию о видео (id канала, заголовок, описание).

In [35]:
response_search['items'][0]['id']['videoId']

'zsYIw6RXjfM'

К слову, само видео можно посмотреть так: https://www.youtube.com/watch?v=zsYIw6RXjfM. Обратите внимание, куда именно подставлятся наше id.

## Комментарии к определенному видео

Например, можно забрать комментарии определенного видео (а в перспективе — посмотреть их тональность). Поможет нам в этом метод `.commentThreads()`.

In [36]:
comment_request = youtube.commentThreads().list(
    part = 'snippet',
    videoId = "zsYIw6RXjfM"
)

comment_response = comment_request.execute()
comment_response

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'doWym_XIduHgIRKNZIhQ_Qc01ZA',
 'nextPageToken': 'Z2V0X25ld2VzdF9maXJzdC0tQ2dnSWdBUVZGN2ZST0JJRkNKMGdHQUVTQlFpSklCZ0FFZ1VJaHlBWUFCSUZDSWdnR0FBWUFDSU9DZ3dJZzdPcXB3WVE2SXVmamdF',
 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20},
 'items': [{'kind': 'youtube#commentThread',
   'etag': 'vUjDZE1NIvVuSpN2ZN9kiK19i8Y',
   'id': 'Ugwv2of8WoJe24fHanB4AaABAg',
   'snippet': {'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
    'videoId': 'zsYIw6RXjfM',
    'topLevelComment': {'kind': 'youtube#comment',
     'etag': '-wQv1t3MJ1XQJ3yOPDrlZPND5wo',
     'id': 'Ugwv2of8WoJe24fHanB4AaABAg',
     'snippet': {'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
      'videoId': 'zsYIw6RXjfM',
      'textDisplay': 'Rapidly build and deploy APIs in seconds with DreamFactory ! <a href="https://bit.ly/3M1TAP7">https://bit.ly/3M1TAP7</a>',
      'textOriginal': 'Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7',
      'authorDispla

In [37]:
for comment in comment_response['items']:
    print(comment['snippet']['topLevelComment']['snippet']['textOriginal'], '\n')

Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7 

how to open dream factory admin page? 

PLEASE SOMEONE HELP WHAT iS HE USNG iS THAT REPLiT?  AND iF SO HOW DO i PULL UP THAT TERMiNAL ALL i HAVE iS SHELL AND SHELL DOESNT LET ME PASTE ANTHiNG 

Thank you sir! 

just a note: when you call the api in the url, you do not need to put "extra" between quotes. ;) 

At 8:33, I'm getting a 404 error. My url stops at :5000 and doesn't list 'get-user' etc. I went through the code letter by letter and can't find what I might be missing. Anyone have any ideas? 

Cam someone explain m3 why do we do that? 

What keyboard do you have? 

The delivery of the information was very clear, but I had difficulty understanding the "why" behind a lot of things you did in the video. 

Youre superior for this unimaginable website hosting video! Ive experimented with Hostwinds and HostGator, however Cloudways with TST20 coupon is the best money saver. 

Youre superior for this un

## Статистика по определенному видео
Последний метод в этом погружении в API YouTube — `.videos()`

In [38]:
video_request = youtube.videos().list(
    part = ['snippet', 'contentDetails', 'id', 'statistics'],
    id = 'zsYIw6RXjfM'
)

video_response = video_request.execute()
video_response

{'kind': 'youtube#videoListResponse',
 'etag': 'zy0w40A8Gw3D2wlxM5vl6jlfd0c',
 'items': [{'kind': 'youtube#video',
   'etag': 'GdKLQV4gYV9K2n-1sIASK_Uq0_k',
   'id': 'zsYIw6RXjfM',
   'snippet': {'publishedAt': '2023-05-23T13:00:53Z',
    'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
    'title': 'Create A Python API in 12 Minutes',
    'description': 'Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7\n\nAre you learning programming and wondering what an API is? An API is an Application Programming Interface that allows different software or systems to communicate with one another. I\'ll show you how to whip up your very own API in just 10 minutes using Python and Flask. \n\nPost Man API Platform: \nhttps://www.postman.com/\n\nFix Pip (Windows): \nhttps://www.youtube.com/watch?v=AdUZArA-kZw&t=204s\n\nFix Pip (Mac): \nhttps://www.youtube.com/watch?v=E-WhAS6qzsU\n\n💻 Master Blockchain and Web 3.0 development today by using BlockchainExpert: 🔗 https://algoexpe

Теперь попробуем собрать информацию о всех наших десяти видео!

In [39]:
video_ids = []

for video in response_search['items']:
    video_ids.append(video['id']['videoId'])

print(video_ids)

['zsYIw6RXjfM', '0sOvCWFmrtA', 'qbLc5a9jdXo', '2HfSFdPEFRg', 'hpc5jyVpUpw', 'bKCORrHbutQ', 'th5_9woFJmk', '5ZMpbdK0uqU', 'F43rgxq4CKw', 'WXsD0ZgxjRw']


In [40]:
video_10_request = youtube.videos().list(
    part = ['snippet', 'contentDetails', 'id', 'statistics'],
    id = video_ids
)

video_10_response = video_10_request.execute()

In [41]:
video_10_response

{'kind': 'youtube#videoListResponse',
 'etag': '6daPXr0thYseoDM9y4PzYbKFsQQ',
 'items': [{'kind': 'youtube#video',
   'etag': 'BEAPubE8Dk4BtsKMyWd57V89wKk',
   'id': 'zsYIw6RXjfM',
   'snippet': {'publishedAt': '2023-05-23T13:00:53Z',
    'channelId': 'UC4JX40jDee_tINbkjycV4Sg',
    'title': 'Create A Python API in 12 Minutes',
    'description': 'Rapidly build and deploy APIs in seconds with DreamFactory ! https://bit.ly/3M1TAP7\n\nAre you learning programming and wondering what an API is? An API is an Application Programming Interface that allows different software or systems to communicate with one another. I\'ll show you how to whip up your very own API in just 10 minutes using Python and Flask. \n\nPost Man API Platform: \nhttps://www.postman.com/\n\nFix Pip (Windows): \nhttps://www.youtube.com/watch?v=AdUZArA-kZw&t=204s\n\nFix Pip (Mac): \nhttps://www.youtube.com/watch?v=E-WhAS6qzsU\n\n💻 Master Blockchain and Web 3.0 development today by using BlockchainExpert: 🔗 https://algoexpe

## Базовая обработка выгрузки из API

Чтобы привести это все в относительно приличный вид, сделаем таблицу. Поможет нам в этом модуль pandas, нам понадобится только один метод — `json_normalize()`.

In [42]:
import pandas as pd
table = pd.json_normalize(video_10_response['items'])
table

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,contentDetails.duration,contentDetails.dimension,contentDetails.definition,contentDetails.caption,contentDetails.licensedContent,contentDetails.projection,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount
0,youtube#video,BEAPubE8Dk4BtsKMyWd57V89wKk,zsYIw6RXjfM,2023-05-23T13:00:53Z,UC4JX40jDee_tINbkjycV4Sg,Create A Python API in 12 Minutes,Rapidly build and deploy APIs in seconds with ...,https://i.ytimg.com/vi/zsYIw6RXjfM/default.jpg,120,90,...,PT12M5S,2d,hd,true,True,rectangular,347189,7916,0,105
1,youtube#video,uvQd05zicVlTNI-bfefsRTPhgTE,0sOvCWFmrtA,2021-11-01T13:11:23Z,UC8butISFwT-Wl7EV0hUK0BQ,Python API Development - Comprehensive Course ...,Learn Python API development in one of the mos...,https://i.ytimg.com/vi/0sOvCWFmrtA/default.jpg,120,90,...,PT19H27S,2d,hd,true,True,rectangular,1851646,47686,0,1927
2,youtube#video,u5c5LU1UdkpZNBCfwntZI6ltYyw,qbLc5a9jdXo,2020-11-19T20:27:58Z,UCZUyPT9DkJWmS_DzdOi7RIA,REST API Crash Course - Introduction + Full Py...,Start your software dev career - https://calcu...,https://i.ytimg.com/vi/qbLc5a9jdXo/default.jpg,120,90,...,PT52M50S,2d,hd,false,True,rectangular,811982,19298,0,648
3,youtube#video,ie6dnNoOslekNDl3V1PS2SnWPPw,2HfSFdPEFRg,2022-05-31T10:30:01Z,UC7cs8q-gJRlGwj4A8OmCmXg,How to use a Public API | Using a Public API w...,APIs are extremely useful to know how to use. ...,https://i.ytimg.com/vi/2HfSFdPEFRg/default.jpg,120,90,...,PT14M1S,2d,hd,false,True,rectangular,25370,952,0,60
4,youtube#video,sict3XqELfuc_RkHaIH1DNHu6iY,hpc5jyVpUpw,2019-11-02T14:00:09Z,UCF1dnm4HfA9LYQo9fVU5RKQ,Working with APIs in Python - Code in 10 Minutes,"In this video, I teach you the basics of APIs ...",https://i.ytimg.com/vi/hpc5jyVpUpw/default.jpg,120,90,...,PT8M41S,2d,hd,false,True,rectangular,279112,4762,0,113
5,youtube#video,Aifby3BbrU8LJxRUzmQ8ena-_XA,bKCORrHbutQ,2021-04-19T03:32:37Z,UCO3rFwdoi0sk1RKP2xjhUtg,How to Fetch APIs with Python | API Fetching W...,"Hey guys! Welcome back! In this video, I'll be...",https://i.ytimg.com/vi/bKCORrHbutQ/default.jpg,120,90,...,PT7M32S,2d,hd,false,True,rectangular,71329,900,0,18
6,youtube#video,K0HYdB-C6aF5jo2rl0AvYt3B1RE,th5_9woFJmk,2020-05-29T16:17:07Z,UCCezIgC97PvUuR4_gbFUs5g,Python YouTube API Tutorial: Getting Started -...,"In this Python Programming Tutorial, we'll be ...",https://i.ytimg.com/vi/th5_9woFJmk/default.jpg,120,90,...,PT20M49S,2d,hd,false,True,rectangular,196830,5583,0,329
7,youtube#video,0f2oCUILE4HSaXLlOMeGT7H61sE,5ZMpbdK0uqU,2021-01-27T08:30:00Z,UCuudpdbKmQWq2PPzYgVCWlA,Create your own Public API in Python Tutorial,Hey guys! In this video I'll be showing you ho...,https://i.ytimg.com/vi/5ZMpbdK0uqU/default.jpg,120,90,...,PT8M37S,2d,hd,false,True,rectangular,162245,2566,0,63
8,youtube#video,D1r8FGzhqmfRfhM7OIRLOQ2a4-w,F43rgxq4CKw,2023-02-04T17:00:08Z,UCuudpdbKmQWq2PPzYgVCWlA,How To Create Your Very First API With FastAPI...,"In this tutorial, you will learn how to create...",https://i.ytimg.com/vi/F43rgxq4CKw/default.jpg,120,90,...,PT10M18S,2d,hd,false,True,rectangular,46550,1286,0,73
9,youtube#video,T7urwXiUjmDhYOOlM7nwjs2yMWs,WXsD0ZgxjRw,2023-02-06T15:53:28Z,UC8butISFwT-Wl7EV0hUK0BQ,APIs for Beginners 2023 - How to use an API (F...,What is an API? Learn all about APIs (Applicat...,https://i.ytimg.com/vi/WXsD0ZgxjRw/default.jpg,120,90,...,PT3H7M7S,2d,hd,true,True,rectangular,1217399,25999,0,563


In [43]:
table['statistics.viewCount'] = pd.to_numeric(table['statistics.viewCount'])

In [44]:
table.sort_values(by = 'statistics.viewCount', ascending = False)

,kind,etag,id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,snippet.thumbnails.default.height,...,contentDetails.duration,contentDetails.dimension,contentDetails.definition,contentDetails.caption,contentDetails.licensedContent,contentDetails.projection,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount
1,youtube#video,uvQd05zicVlTNI-bfefsRTPhgTE,0sOvCWFmrtA,2021-11-01T13:11:23Z,UC8butISFwT-Wl7EV0hUK0BQ,Python API Development - Comprehensive Course ...,Learn Python API development in one of the mos...,https://i.ytimg.com/vi/0sOvCWFmrtA/default.jpg,120,90,...,PT19H27S,2d,hd,true,True,rectangular,1851646,47686,0,1927
9,youtube#video,T7urwXiUjmDhYOOlM7nwjs2yMWs,WXsD0ZgxjRw,2023-02-06T15:53:28Z,UC8butISFwT-Wl7EV0hUK0BQ,APIs for Beginners 2023 - How to use an API (F...,What is an API? Learn all about APIs (Applicat...,https://i.ytimg.com/vi/WXsD0ZgxjRw/default.jpg,120,90,...,PT3H7M7S,2d,hd,true,True,rectangular,1217399,25999,0,563
2,youtube#video,u5c5LU1UdkpZNBCfwntZI6ltYyw,qbLc5a9jdXo,2020-11-19T20:27:58Z,UCZUyPT9DkJWmS_DzdOi7RIA,REST API Crash Course - Introduction + Full Py...,Start your software dev career - https://calcu...,https://i.ytimg.com/vi/qbLc5a9jdXo/default.jpg,120,90,...,PT52M50S,2d,hd,false,True,rectangular,811982,19298,0,648
0,youtube#video,BEAPubE8Dk4BtsKMyWd57V89wKk,zsYIw6RXjfM,2023-05-23T13:00:53Z,UC4JX40jDee_tINbkjycV4Sg,Create A Python API in 12 Minutes,Rapidly build and deploy APIs in seconds with ...,https://i.ytimg.com/vi/zsYIw6RXjfM/default.jpg,120,90,...,PT12M5S,2d,hd,true,True,rectangular,347189,7916,0,105
4,youtube#video,sict3XqELfuc_RkHaIH1DNHu6iY,hpc5jyVpUpw,2019-11-02T14:00:09Z,UCF1dnm4HfA9LYQo9fVU5RKQ,Working with APIs in Python - Code in 10 Minutes,"In this video, I teach you the basics of APIs ...",https://i.ytimg.com/vi/hpc5jyVpUpw/default.jpg,120,90,...,PT8M41S,2d,hd,false,True,rectangular,279112,4762,0,113
6,youtube#video,K0HYdB-C6aF5jo2rl0AvYt3B1RE,th5_9woFJmk,2020-05-29T16:17:07Z,UCCezIgC97PvUuR4_gbFUs5g,Python YouTube API Tutorial: Getting Started -...,"In this Python Programming Tutorial, we'll be ...",https://i.ytimg.com/vi/th5_9woFJmk/default.jpg,120,90,...,PT20M49S,2d,hd,false,True,rectangular,196830,5583,0,329
7,youtube#video,0f2oCUILE4HSaXLlOMeGT7H61sE,5ZMpbdK0uqU,2021-01-27T08:30:00Z,UCuudpdbKmQWq2PPzYgVCWlA,Create your own Public API in Python Tutorial,Hey guys! In this video I'll be showing you ho...,https://i.ytimg.com/vi/5ZMpbdK0uqU/default.jpg,120,90,...,PT8M37S,2d,hd,false,True,rectangular,162245,2566,0,63
5,youtube#video,Aifby3BbrU8LJxRUzmQ8ena-_XA,bKCORrHbutQ,2021-04-19T03:32:37Z,UCO3rFwdoi0sk1RKP2xjhUtg,How to Fetch APIs with Python | API Fetching W...,"Hey guys! Welcome back! In this video, I'll be...",https://i.ytimg.com/vi/bKCORrHbutQ/default.jpg,120,90,...,PT7M32S,2d,hd,false,True,rectangular,71329,900,0,18
8,youtube#video,D1r8FGzhqmfRfhM7OIRLOQ2a4-w,F43rgxq4CKw,2023-02-04T17:00:08Z,UCuudpdbKmQWq2PPzYgVCWlA,How To Create Your Very First API With FastAPI...,"In this tutorial, you will learn how to create...",https://i.ytimg.com/vi/F43rgxq4CKw/default.jpg,120,90,...,PT10M18S,2d,hd,false,True,rectangular,46550,1286,0,73
3,youtube#video,ie6dnNoOslekNDl3V1PS2SnWPPw,2HfSFdPEFRg,2022-05-31T10:30:01Z,UC7cs8q-gJRlGwj4A8OmCmXg,How to use a Public API | Using a Public API w...,APIs are extremely useful to know how to use. ...,https://i.ytimg.com/vi/2HfSFdPEFRg/default.jpg,120,90,...,PT14M1S,2d,hd,false,True,rectangular,25370,952,0,60
